In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Model
import utils

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [ ]:
if "maler" in str(Path(".").resolve()):
    absolute_path = Path("C:/Users/maler/Federico/Lavoro/ZIB/experiments_results") 
elif "luziehel" in str(Path(".").resolve()):
    absolute_path = Path("C:/Users/luziehel/Code/experiments_results")

series_dir = absolute_path.joinpath("2_opt-h_luzie-gamma_delta_expl")
series_dir = absolute_path.joinpath("2_opt-h_luzie-alpha_beta_gamma_delta_expl")
series_dir = absolute_path.joinpath("2_opt-h_luzie-alpha_beta_gamma_delta_expl-0.0001t")

experiment_runs_folders = [folder for folder in series_dir.iterdir()]

model_path = Path("C:/Users/maler/Federico/Lavoro/ZIB/autoencoder_clustering/autoencoder_model/model.h5")

In [ ]:
model = tf.keras.models.load_model(model_path)
autoencoder = Model(model.input, model.get_layer('embedding').output)

In [ ]:
folder = np.random.choice(experiment_runs_folders)
state_metric = utils.load_metrics(folder)["StateMetric"][::4]
inputs = state_metric.flatten()
assert np.all(np.reshape(inputs, state_metric.shape) == state_metric)

preds = model.predict(inputs[np.newaxis])

In [ ]:
utils.plot_agents_option(state_metric)

In [ ]:
utils.plot_agents_option(np.reshape(preds, state_metric.shape))

# Random plots

In [ ]:
folders = np.random.choice(experiment_runs_folders, size=10)
state_metrics = [utils.load_metrics(folder)["StateMetric"][::4] for folder in folders]
inputs = np.array([state_metric.flatten() for state_metric in state_metrics])

preds = model.predict(inputs)

In [ ]:
n_agents, n_options = state_metrics[0].shape[1:]
    
plt.figure(figsize=(20, 60))
for i in range(len(state_metrics)):

    for option in range(n_options):
        plt.subplot(n_options*len(state_metrics), 2, 1 + i*4 + option*2)
        for agent in range(n_agents):
            plt.plot(state_metrics[i][:, agent, option], label=str(agent))
            #plt.legend()
            #plt.title("Option "+ str(option+1))
    for option in range(n_options):
        plt.subplot(n_options*len(state_metrics), 2, 2 + i*4 + option*2)
        for agent in range(n_agents):
            to_plot = np.reshape(preds[i], state_metrics[0].shape)
            plt.plot(to_plot[:, agent, option], label=str(agent))

plt.tight_layout()


plt.show()